In [19]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
mt_path = "/home/cs/yangyuchen/guoyiqiu/kg_llm/output/llama_chat_usmle_kg_0.2"
model = AutoModelForCausalLM.from_pretrained(mt_path).half().cuda()
tok = AutoTokenizer.from_pretrained(mt_path)
tok.pad_token = tok.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
data_path = "data/kg_chat_usmle_10178.json"
kg_path = "data/umls_kg_filter_count_5_with_def_len_100.csv"
kg = pd.read_csv(kg_path)
data = json.load(open(data_path))
all_triplets = [tid for i in data for triplets in i['input_triplets']+i['output_triplets'] for tid in triplets]
all_triplets = list(set(all_triplets))
all_triplets = [kg.iloc[tid] for tid in all_triplets]
print(len(all_triplets))
from collections import defaultdict
edge_count = defaultdict(int)
for t in all_triplets:
    edge_count[t.edge] += 1
print(edge_count)

6212
defaultdict(<class 'int'>, {'has definition of': 5860, 'possibly equivalent to': 156, 'has occurrence': 60, 'has direct substance': 15, 'has causative agent': 65, 'has direct procedure site': 4, 'has direct morphology': 4, 'has active ingredient': 10, 'has associated morphology': 30, 'has procedure site': 3, 'has finding site': 5})


In [21]:
from tqdm.auto import tqdm
import fastlcs 
relationship_templates = json.load(open('data/relationship_templates.json'))
PROMPT_TEMPLATE = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\n\n##USER:\n{input}\n\n##ASSISTANT:\n{output}"
for t in tqdm(all_triplets[0:]):
    tmps = relationship_templates[t.edge]
    lcs_scores = 0
    gt_text = t.target
    gt_len = len(tok(gt_text)['input_ids'])
    print('\ngt text: ', gt_text)
    for tmp in tmps:
        input_text = input=tmp.format(t.source)
        # input_text = PROMPT_TEMPLATE.format(input=tmp.format(t.source), output='')
        print('question: ', tmp.format(t.source))
        inp = tok(input_text, return_tensors='pt')
        res = model.generate(input_ids=inp['input_ids'].cuda(), attention_mask=inp['attention_mask'].cuda(), max_new_tokens=gt_len*2)
        output_text = tok.decode(res[0,inp['input_ids'].shape[-1]:])
        lcs_length = fastlcs.lcsubstr_length(gt_text,output_text)
        lcs_scores += lcs_length/len(gt_text)/len(tmps)
        # print('lcsubstr: ', lcsubstr)
        # print('lcs_score: ', lcsubstr/len(gt_text))
        print('output_text: ', output_text)
    print('lcs_scores: ', lcs_scores)

  0%|          | 0/6212 [00:00<?, ?it/s]


gt text:  Inflammation of the body fundic mucosa of the stomach. It results from the development of autoantibodies against the parietal and chief cells. It is associated with the presence of intestinal metaplasia and an increased risk of developing gastric carcinoma.
question:  Autoimmune Gastritis has definition of


/home/cs/yangyuchen/miniconda3/envs/med_llm/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


output_text:  Inflammation of the GASTRIC MUCOSA due to autoimmune responses. It is often seen as a result of a chronic autoimmune process such as SLAVIN DISEASE. Symptoms include loss of appetite, abdominal pain, and weight loss. (From Harrison's Principles of Internal Medicine, 12th ed) can occur in patients with SLAVIN DISEASE. (From Harrison's Principles of Internal Medicine, 12th ed) (autoimmune gastritis) (disorder) (gastritis) (
question:  The definition of Autoimmune Gastritis is
output_text:  an autoimmune disorder characterized by destruction of the parietal cells in the gastric mucosa. is characterized by destruction of the parietal cells in the gastric mucosa. is characterized by destruction of the parietal cells in the gastric mucosa. is characterized by destruction of the parietal cells in the gastric mucosa. is characterized by destruction of the parietal cells in the gastric mucosa. is characterized by destruction of the parietal cells in the gastric mucosa. is characte